In [1]:
import sleap_io as sio
# import sleap
import pandas as pd

In [13]:
def convert_predictions_to_dataframe(labels) -> pd.DataFrame:
    """Helper function to convert predictions data to a Pandas dataframe.

    Args:
        labels: A SLEAP Labels object.

    Raises:
        ValueError: If no frames in the label objects contain predicted instances.

    Returns:
        pd.DataFrame: A pandas data frame with the structured data with
        hierarchical columns. The column hierarchy is:
                "video_path",
                "skeleton_name",
                "track_name",
                "node_name",
        And it is indexed by the frames.
    """

    # Form pairs of labeled_frames and predicted instances
    labeled_frames = labels.labeled_frames
    all_frame_instance_tuples = (
        (label_frame, instance)  # type: ignore
        for label_frame in labeled_frames
        for instance in label_frame.predicted_instances
    )

    # Extract the data
    data_list = list()
    for labeled_frame, instance in all_frame_instance_tuples:
        # Traverse the nodes of the instances's skeleton
        skeleton = instance.skeleton
        for node in skeleton.nodes:
            row_dict = dict(
                frame_idx=labeled_frame.frame_idx,
                x=instance.points[node].x,
                y=instance.points[node].y,
                score=instance.points[node].score,  # type: ignore[attr-defined]
                node_name=node.name,
                skeleton_name=skeleton.name,
                track_name=instance.track.name if instance.track else "untracked",
                video_path=labeled_frame.video.filename,
            )
            data_list.append(row_dict)

    if not data_list:
        raise ValueError("No predicted instances found in labels object")

    labels_df = pd.DataFrame(data_list)

    # Reformat the data with columns for dict-like hierarchical data access.
    index = [
        "skeleton_name",
        "track_name",
        "node_name",
        "video_path",
        "frame_idx",
    ]

    labels_tidy_df = (
        labels_df.set_index(index)
        .unstack(level=[0, 1, 2, 3])
        .swaplevel(0, -1, axis=1)  # video_path on top while x, y score on bottom
        .sort_index(axis=1)  # Better format for columns
        .sort_index(axis=0)  # Sorts by frames
    )

    return labels_tidy_df

In [2]:
labels = sio.load_slp(r"D:\code\wt_gold-SR\data\190612_110405_wt_16276625_rig2.1\inference.cleaned.proofread.slp")

Labels(labeled_frames=270095, videos=1, skeletons=1, tracks=2)

In [14]:
df = convert_predictions_to_dataframe(labels)

In [15]:
df.sample(n=5)

video_path    T:/junyu/data/pair/wt/190612_110405_wt_16276625_rig2.1\000000.mp4  \
skeleton_name                                                        Skeleton-3   
track_name                                                              track_0   
node_name                                                               abdomen   
                                                                          score   
frame_idx                                                                         
260804                                                  0.932082                  
40486                                                   0.860933                  
247737                                                  0.674291                  
12724                                                   0.815026                  
68788                                                   0.792495                  

video_path                                                                 \
skeleton_name                                                               
track_name                                                                  
node_name                          eyeL                      eyeR           
                    x       y     score       x       y     score       x   
frame_idx                                                                   
260804         494.75  535.25  0.947798  480.25  468.25  0.914947  502.75   
40486          472.75  212.25  0.895753  487.25  278.75  0.958784  465.25   
247737         461.25  504.25  0.982686  528.75  479.25  0.922243  532.25   
12724          218.25  686.75  0.618205  164.75  637.75  0.694839  178.75   
68788          472.75  213.75  0.906278  488.75  280.75  0.947246  466.75   

video_path                       ...                                     \
skeleton_name                    ...                                      
track_name                       ...  track_1                             
node_name             forelegL4  ... midlegR4    thorax                   
                    y     score  ...        y     score       x       y   
frame_idx                        ...                                      
260804         465.75  0.840839  ...   455.75  0.962049  406.25  413.25   
40486          280.25  1.000069  ...   638.75  0.890574  403.75  630.75   
247737         501.25  0.974580  ...   530.75  1.040926  615.75  563.25   
12724          621.75  0.716039  ...   659.75  1.000365  313.75  697.75   
68788          282.25  1.030797  ...   228.25  0.941238  594.75  259.25   

video_path                                                         
skeleton_name                                                      
track_name                                                         
node_name         wingL                     wingR                  
                  score       x       y     score       x       y  
frame_idx                                                          
260804         0.947129  363.25  387.75  0.876978  361.75  391.25  
40486          0.948809  393.75  682.25  0.984388  415.25  681.75  
247737         0.968004  658.75  587.25  1.010607  667.75  575.75  
12724          1.019276  360.25  715.25  0.796638  361.25  711.25  
68788          1.064914  644.75  273.25  1.075285  645.75  253.25  

[5 rows x 78 columns]

In [19]:
df.columns.levels

FrozenList([['T:/junyu/data/pair/wt/190612_110405_wt_16276625_rig2.1\000000.mp4'], ['Skeleton-3'], ['track_0', 'track_1'], ['abdomen', 'eyeL', 'eyeR', 'forelegL4', 'forelegR4', 'head', 'hindlegL4', 'hindlegR4', 'midlegL4', 'midlegR4', 'thorax', 'wingL', 'wingR'], ['score', 'x', 'y']])